# 시드값 고정 및 GPU 장비 설정
## 1.시드값 고정

In [1]:
import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

## 2.GPU 장비 설정

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# 데이터 준비
## 1.데이터 불러오기

In [3]:
data_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/'
train_path = data_path + 'train/'
valid_path = data_path + 'val/'
test_path = data_path + 'test/'

## 2.이미지 변환기 정의(데이터 증강)

In [4]:
from torchvision import transforms

# 훈련 데이터용 변환기
transform_train = transforms.Compose([
    transforms.Resize((250, 250)),                                     # 이미지 크기 조정
    transforms.RandomHorizontalFlip(0.5),                              # 좌우 대칭
    transforms.RandomVerticalFlip(0.2),                                # 상하 대칭
    transforms.RandomRotation(20),                                     # 이미지 회전
    transforms.ToTensor(),                                             # 텐서 객체로 변환
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # 정규화
])

# 테스트 데이터용 변환기
transform_test = transforms.Compose([
    transforms.Resize((250, 250)),                                     # 이미지 크기 조정
    transforms.CenterCrop(180),                                        # 가운데 부분 자르기
    transforms.ToTensor(),                                             # 텐서 객체로 변환
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # 정규화
])

## 3.데이터셋 생성

In [5]:
from torchvision.datasets import ImageFolder

datasets_train = ImageFolder(root=train_path, transform=transform_train) # 훈련 데이터셋
datasets_valid = ImageFolder(root=valid_path, transform=transform_test)  # 검증 데이터셋
datasets_test = ImageFolder(root=test_path, transform=transform_test)    # 테스트 데이터셋

In [6]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

## 4.데이터 로더 생성

In [7]:
from torch.utils.data import DataLoader

batch_size = 8

loader_train = DataLoader(dataset=datasets_train, batch_size=batch_size, shuffle=True,
                          worker_init_fn=seed_worker, generator=g, num_workers=2)
loader_valid = DataLoader(dataset=datasets_valid, batch_size=batch_size, shuffle=False,
                          worker_init_fn=seed_worker, generator=g, num_workers=2)
loader_test = DataLoader(dataset=datasets_test, batch_size=batch_size, shuffle=False,
                          worker_init_fn=seed_worker, generator=g, num_workers=2)

# 모델 생성

In [8]:
!pip install efficientnet-pytorch==0.7.1

In [9]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)
model = model.to(device)

Loaded pretrained weights for efficientnet-b0


In [10]:
print('모델 파라미터 개수:', sum(param.numel() for param in model.parameters()))

모델 파라미터 개수: 4010110


# 모델 훈련 및 성능 검증
## 1.손실함수와 옵티마이저 설정

In [11]:
import torch.nn as nn

# 손실함수
criterion = nn.CrossEntropyLoss()

In [12]:
# 옵티마이저
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

## 2.훈련 함수 작성

In [13]:
from sklearn.metrics import accuracy_score # 정확도 계산 함수
from sklearn.metrics import recall_score   # 재현율 계산 함수
from sklearn.metrics import f1_score       # F1 점수 계산 함수
from tqdm.notebook import tqdm             # 진행률 표시 막대

def train(model, loader_train, loader_valid, criterion, optimizer, 
          scheduler=None, epochs=10, save_file='model_state_dict.pth'):
    valid_loss_min = np.inf # 최소 손실값 초기화(검증 데이터용) 

    for epoch in range(epochs):
        print(f'에포크 [{epoch+1}/{epochs}] \n-----------------------------')
        
        # == [ 훈련 ] ==============================================
        model.train()
        epoch_train_loss = 0 
        
        for images, labels in tqdm(loader_train):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_train_loss += loss.item() 
            loss.backward()
            optimizer.step()  
            if scheduler != None:
                scheduler.step() 

        print(f'\t훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
        
        # == [ 검증 ] ==============================================
        model.eval()         
        epoch_valid_loss = 0
        preds_list = []
        true_list = []    
        
        with torch.no_grad():
            for images, labels in loader_valid:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_valid_loss += loss.item()
                preds = torch.max(outputs.cpu(), dim=1)[1].numpy() 
                true = labels.cpu().numpy() 
                preds_list.extend(preds)
                true_list.extend(true)
        
        val_accuracy = accuracy_score(true_list, preds_list) # 정확도
        val_recall = recall_score(true_list, preds_list)     # 재현율
        val_f1_score = f1_score(true_list, preds_list)       # F1 점수
        print(f'\t검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')
        print(f'\t정확도 : {val_accuracy:.4f} / 재현율 : {val_recall:.4f} / F1 점수 : {val_f1_score:.4f}')
        
        # == [ 최적 모델 가중치 찾기 ] ==============================
        if epoch_valid_loss <= valid_loss_min: 
            print(f'\t### 검증 데이터 손실값 감소 ({valid_loss_min:.4f} -> {epoch_valid_loss:.4f}). 모델 저장')
            torch.save(model.state_dict(), save_file) 
            valid_loss_min = epoch_valid_loss
            
    return torch.load(save_file)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 3.훈련 및 성능 검증

In [14]:
# 모델 훈련
model_state_dict = train(model=model,
                         loader_train=loader_train, 
                         loader_valid=loader_valid,
                         criterion=criterion,
                         optimizer=optimizer)

에포크 [1/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.3931
	검증 데이터 손실값 : 3.3658
	정확도 : 0.5000 / 재현율 : 0.8750 / F1 점수 : 0.6364
	### 검증 데이터 손실값 감소 (inf -> 6.7317). 모델 저장
에포크 [2/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.2418
	검증 데이터 손실값 : 1.4240
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667
	### 검증 데이터 손실값 감소 (6.7317 -> 2.8481). 모델 저장
에포크 [3/10] 
-----------------------------


...

	훈련 데이터 손실값 : 0.1556
	검증 데이터 손실값 : 3.5117
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667
에포크 [9/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.1401
	검증 데이터 손실값 : 3.2903
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667
에포크 [10/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.1410
	검증 데이터 손실값 : 2.1733
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667


In [15]:
# 최적 가중치 불러오기
model.load_state_dict(model_state_dict)

<All keys matched successfully>

# 예측 및 평가 결과
## 1.예측

In [16]:
def predict(model, loader_test, return_true=False):
    model.eval()    # 모델을 평가 상태로 설정
    preds_list = [] # 예측값 저장용 리스트 초기화
    true_list = []  # 실젯값 저장용 리스트 초기화
    
    with torch.no_grad():
        for images, labels in loader_test:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            preds = torch.max(outputs.cpu(), dim=1)[1].numpy()
            true = labels.cpu().numpy()
            preds_list.extend(preds)
            true_list.extend(true)
            
    if return_true:
        return true_list, preds_list
    else:
        return preds_list

In [17]:
true_list, preds_list = predict(model=model, loader_test=loader_test, return_true=True)

## 2.평가 결과

In [18]:
print('##### 최종 예측 결과 평가 점수 #####')
print(f'정확도: {accuracy_score(true_list, preds_list):.4f}')
print(f'재현율: {recall_score(true_list, preds_list):.4f}')
print(f'F1 점수: {f1_score(true_list, preds_list):.4f}')

##### 최종 예측 결과 평가 점수 #####
정확도: 0.7340
재현율: 0.9769
F1 점수: 0.8211


# 성능 개선
## 1.모델 생성

In [19]:
# 모델 저장용 리스트
models_list = []

# 모델 생성
efficientnet_b1 = EfficientNet.from_pretrained('efficientnet-b1', num_classes=2)
efficientnet_b2 = EfficientNet.from_pretrained('efficientnet-b2', num_classes=2)
efficientnet_b3 = EfficientNet.from_pretrained('efficientnet-b3', num_classes=2)

# 장비 할당
efficientnet_b1 = efficientnet_b1.to(device)
efficientnet_b2 = efficientnet_b2.to(device)
efficientnet_b3 = efficientnet_b3.to(device)

# 리스트에 모델 저장
models_list.append(efficientnet_b1)
models_list.append(efficientnet_b2)
models_list.append(efficientnet_b3)

Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3


In [20]:
for idx, model in enumerate(models_list):
    num_params = sum(param.numel() for param in model.parameters())
    print(f'모델{idx+1} 파라미터 개수: {num_params}')

모델1 파라미터 개수: 6515746
모델2 파라미터 개수: 7703812
모델3 파라미터 개수: 10699306


## 2.손실함수, 옵티마이저, 스케줄러 설정

In [21]:
# 손실함수
criterion = nn.CrossEntropyLoss()

In [22]:
# 옵티마이저
optimizer1 = torch.optim.AdamW(models_list[0].parameters(), lr=0.0006, weight_decay=0.001)
optimizer2 = torch.optim.AdamW(models_list[1].parameters(), lr=0.0006, weight_decay=0.001)
optimizer3 = torch.optim.AdamW(models_list[2].parameters(), lr=0.0006, weight_decay=0.001)

In [23]:
from transformers import get_cosine_schedule_with_warmup

epochs = 10

# 스케줄러
scheduler1 = get_cosine_schedule_with_warmup(optimizer1, 
                                             num_warmup_steps=len(loader_train)*3,
                                             num_training_steps=len(loader_train)*epochs)
scheduler2 = get_cosine_schedule_with_warmup(optimizer2, 
                                             num_warmup_steps=len(loader_train)*3,
                                             num_training_steps=len(loader_train)*epochs)
scheduler3 = get_cosine_schedule_with_warmup(optimizer3, 
                                             num_warmup_steps=len(loader_train)*3,
                                             num_training_steps=len(loader_train)*epochs)

## 3.훈련 및 성능 검증

In [24]:
# 첫번째 모델 훈련
model_state_dict = train(model=models_list[0],
                         loader_train=loader_train, 
                         loader_valid=loader_valid,
                         criterion=criterion,
                         optimizer=optimizer1,
                         scheduler=scheduler1,
                         epochs=epochs)

# 첫번째 모델에 최적 가중치 적용
models_list[0].load_state_dict(model_state_dict)

에포크 [1/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.2843
	검증 데이터 손실값 : 0.4822
	정확도 : 0.7500 / 재현율 : 1.0000 / F1 점수 : 0.8000
	### 검증 데이터 손실값 감소 (inf -> 0.9645). 모델 저장
에포크 [2/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.1278
	검증 데이터 손실값 : 1.0018
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667
에포크 [3/10] 
-----------------------------


...

  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0195
	검증 데이터 손실값 : 3.3596
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667
에포크 [10/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0174
	검증 데이터 손실값 : 3.2291
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667


<All keys matched successfully>

In [25]:
# 두번째 모델 훈련
model_state_dict = train(model=models_list[1],
                         loader_train=loader_train, 
                         loader_valid=loader_valid,
                         criterion=criterion,
                         optimizer=optimizer2,
                         scheduler=scheduler2,
                         epochs=epochs)

# 두번째 모델에 최적 가중치 적용
models_list[1].load_state_dict(model_state_dict)

에포크 [1/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.2877
	검증 데이터 손실값 : 0.4397
	정확도 : 0.6875 / 재현율 : 1.0000 / F1 점수 : 0.7619
	### 검증 데이터 손실값 감소 (inf -> 0.8794). 모델 저장
에포크 [2/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.1291
	검증 데이터 손실값 : 0.8830
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667
에포크 [3/10] 
-----------------------------


...

  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0164
	검증 데이터 손실값 : 3.0586
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667
에포크 [10/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0133
	검증 데이터 손실값 : 2.9544
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667


<All keys matched successfully>

In [26]:
# 세번째 모델 훈련
model_state_dict = train(model=models_list[2],
                         loader_train=loader_train, 
                         loader_valid=loader_valid,
                         criterion=criterion,
                         optimizer=optimizer3,
                         scheduler=scheduler3,
                         epochs=epochs)

# 세번째 모델에 최적 가중치 적용
models_list[2].load_state_dict(model_state_dict)

에포크 [1/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.2657
	검증 데이터 손실값 : 0.4521
	정확도 : 0.6875 / 재현율 : 1.0000 / F1 점수 : 0.7619
	### 검증 데이터 손실값 감소 (inf -> 0.9041). 모델 저장
에포크 [2/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.1172
	검증 데이터 손실값 : 0.5497
	정확도 : 0.6250 / 재현율 : 1.0000 / F1 점수 : 0.7273
에포크 [3/10] 
-----------------------------


...

  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0168
	검증 데이터 손실값 : 2.0927
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667
에포크 [10/10] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0110
	검증 데이터 손실값 : 2.3075
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667


<All keys matched successfully>

## 4.예측 및 평가 결과

In [27]:
# 예측
true_list, preds_list1 = predict(model=models_list[0], loader_test=loader_test, return_true=True)
preds_list2 = predict(model=models_list[1], loader_test=loader_test)
preds_list3 = predict(model=models_list[2], loader_test=loader_test)

In [28]:
# 평가 결과(1)
print('##### efficientnet_b1 모델 예측 결과 평가 점수 #####')
print(f'정확도: {accuracy_score(true_list, preds_list1):.4f}')
print(f'재현율: {recall_score(true_list, preds_list1):.4f}')
print(f'F1 점수: {f1_score(true_list, preds_list1):.4f}')

##### efficientnet_b1 모델 예측 결과 평가 점수 #####
정확도: 0.8157
재현율: 0.9872
F1 점수: 0.8701


In [29]:
# 평가 결과(2)
print('##### efficientnet_b2 모델 예측 결과 평가 점수 #####')
print(f'정확도: {accuracy_score(true_list, preds_list2):.4f}')
print(f'재현율: {recall_score(true_list, preds_list2):.4f}')
print(f'F1 점수: {f1_score(true_list, preds_list2):.4f}')

##### efficientnet_b2 모델 예측 결과 평가 점수 #####
정확도: 0.7051
재현율: 0.9974
F1 점수: 0.8087


In [30]:
# 평가 결과(3)
print('##### efficientnet_b3 모델 예측 결과 평가 점수 #####')
print(f'정확도: {accuracy_score(true_list, preds_list3):.4f}')
print(f'재현율: {recall_score(true_list, preds_list3):.4f}')
print(f'F1 점수: {f1_score(true_list, preds_list3):.4f}')

##### efficientnet_b3 모델 예측 결과 평가 점수 #####
정확도: 0.7788
재현율: 0.9974
F1 점수: 0.8493


## 5.앙상블

In [31]:
# 예측
ensemble_preds = []

for i in range(len(preds_list1)):
    pred_element = np.round((preds_list1[i]+preds_list2[i]+preds_list3[i])/3)
    ensemble_preds.append(pred_element)

In [32]:
# 평가 결과(앙상블)
print('##### 최종 앙상블 예측 결과 평가 점수 #####')
print(f'정확도: {accuracy_score(true_list, ensemble_preds):.4f}')
print(f'재현율: {recall_score(true_list, ensemble_preds):.4f}')
print(f'F1 점수: {f1_score(true_list, ensemble_preds):.4f}')

##### 최종 앙상블 예측 결과 평가 점수 #####
정확도: 0.7612
재현율: 1.0000
F1 점수: 0.8396
